In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from IPython.display import display

# définition des paramètre de la source de données
datapath = "./"
file = "ML_SourceDonnees_diabetes.csv"

# fonction pour la récupération des données
def acquisition_donnees(datapath, file):
    full_path = os.path.join(datapath, file)
    df = pd.read_csv(full_path)
    #afficher les début et fin de données pour vérifier la corruption et l'uniformité des données
    #display(df.head(10))
    #display(df.tail(10))
    #display(df.describe())
    return df    

# fonction qui vérifie s'il existe des données nulles et les affiche 
def null(df):
    pas_de_null = 0
    for i in df:
        if df[i].isnull().values.sum() != 0:
            print(i, df[i])
            pas_de_null = 1
    if pas_de_null == 0:
        print("Pas de null")

# fonction de séparation de la dernière colonne qui représente l'étiquette à prédire
def prep_dataset(df):
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    return X,y

# suppression des valeurs qui ne sont pas des nombres
def suppr_nan(X):
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp_mean.fit(X)
    return imp_mean.transform(X)

# séparation du set de train et du set de test avec des paramètres par défaut
def split(X, y, test_size=0.3, random_state=42): 
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# fonction de normalisation d'une série de données
def normalisation(data):
    scaler = StandardScaler(copy=True)
    scaler.fit(data)
    moy = scaler.mean_
    var = scaler.var_
    return scaler.transform(data)

# méthode de régression Logistique
def regression(X_train,y_train,X_test,y_test):
    regr = linear_model.LogisticRegression()

    # entrainement sur le jeu d'entrainement
    clf = regr.fit(X_train, y_train)

    # prédiction des étiquettes à partir du jeu de test et affichage de la prédiction
    y_pred = clf.predict(X_test)
    print(f"Prediction : {y_pred}")

    # calcul des scores de prédiction
    y_score = clf.score(X_test, y_test)

    auc = roc_auc_score(y_test, y_pred)

    print(f"Score: {y_score:.{4}f}")
    print(f"AUC: {auc:.{4}f}")

### Début du programme ###
# acquisition des données
df = acquisition_donnees(datapath, file)

# détection des valeurs nulles et retrait des valeurs mal définies
null(df)
df_nan = suppr_nan(df)

# préparation du set de datas à partir du dataframe
X,y = prep_dataset(df)
print(X)

# repartition de la partie d'entrainement et de la partie de teste
X_train,X_test,y_train,y_test = split(X,y,0.5,50)

# normalisation des données
X_train_norm = normalisation(X_train)
X_test_norm = normalisation(X_test)

# application de la regression
regression(X_train_norm,y_train,X_test_norm,y_test)

Pas de null
     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0

In [5]:
### Version avec un pipe
from sklearn.pipeline import make_pipeline

full_path = os.path.join(datapath, file)
df = pd.read_csv(full_path)
#display(df.head(10))
#display(df.tail(10))
#display(df.describe())

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

pipe = make_pipeline(StandardScaler(), linear_model.LogisticRegression())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)
pipe.fit(X_train, y_train)
print(pipe.predict(X_test))

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0
 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1
 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0
 1 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
